In [ ]:
import os
import gzip
import json
from pymatgen.io.lobster import Lobsterout
from pymatgen.core import Structure
from pymatgen.electronic_structure.cohp import CompleteCohp
from pymatgen.electronic_structure.dos import LobsterCompleteDos
from atomate2.lobster.schemas import LobsterTaskDocument

In [ ]:
def read_lobstertask_doc_json(filename: str,pymatgen_objs: bool=False):
    """
    This function loads the json.gz files and returns a dictionary with all the lobster outputs.
    
    Args:
        filename (str) : name of file
        pymatgen_objs (bool) : if True will convert structure,coop,cobi,cohp and dos to pymatgen objects
    Returns:
        Returns a dictionary with lobster calculation outputs.
    """
    with gzip.open(filename, 'rb') as f:
        data = json.loads(f.read().decode('utf-8'))
    
    lobster_data = {}
    for item in data:
        lobster_data.update(item)
        
    if pymatgen_objs:
        lobster_data['structure'] = Structure.from_dict(lobster_data['structure'])
        lobster_data['cohp_data'] = CompleteCohp.from_dict(lobster_data['coop_data'])
        lobster_data['cobi_data'] = CompleteCohp.from_dict(lobster_data['cobi_data'])
        lobster_data['coop_data'] = CompleteCohp.from_dict(lobster_data['coop_data'])
        lobster_data['dos'] = LobsterCompleteDos.from_dict(lobster_data['dos'])
        lobster_data['lso_dos'] = LobsterCompleteDos.from_dict(lobster_data['lso_dos'])
        
        for field in ['lobsterpy_data','lobsterpy_data_cation_anion']:
            for plotlabel, cohp in lobster_data[field]["cohp_plot_data"].items():
                lobster_data[field]["cohp_plot_data"][plotlabel]= Cohp.from_dict(cohp)
            
        return lobster_data
    else:
        return lobster_data

In [ ]:
lobtask= read_lobstertask_doc_json(filename='../Example_data/Computational_data_jsons/mp-1000.json.gz', pymatgen_objs=True)